<h1><center>Association Rules</center></h1>

Import library

In [1]:
import pandas as pd 
import numpy as np
import os 
import plotly.graph_objects as go
import plotly.express as px
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

Import Data

In [2]:
path = 'D:\Giáo Trình\Kỳ 5\Kho và Khai Phá Dữ Liệu\Project'
order_data = pd.read_csv(os.path.join(path, 'Data\\Processed Data\\order_data.csv'))
order_data.head()

<>:1: DeprecationWarning: invalid escape sequence \G
<>:1: DeprecationWarning: invalid escape sequence \G
C:\Users\quoct\AppData\Local\Temp\ipykernel_19988\63324331.py:1: DeprecationWarning: invalid escape sequence \G
  path = 'D:\Giáo Trình\Kỳ 5\Kho và Khai Phá Dữ Liệu\Project'


,order_id,order_item_id,customer_unique_id,product_id,seller_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,871766c5855e863f6eccc05f988b23cb,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,eb28e67c4c0b83846050ddfb8a35d051,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15 00:00:00,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,3818d81c6709e39d06b2738a8d3a2474,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05 00:00:00,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,af861d436cfc08b2c2ddefd0ba074622,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20 00:00:00,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,64b576fb70d441e8f1b2d7d446e483c5,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,delivered,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17 00:00:00,199.90,18.14,218.04


In [3]:
product_data = pd.read_csv(os.path.join(path, 'Data\\Processed Data\\product_data.csv'))
product_data.head()

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [4]:
transaction = pd.merge(left = order_data, right = product_data, on = 'product_id', how = 'inner')[['order_id'
                    , 'order_purchase_timestamp', 'product_category_name_english']]
transaction.head()

,order_id,order_purchase_timestamp,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,2017-09-13 08:59:02,cool_stuff
1,130898c0987d1801452a8ed92a670612,2017-06-28 11:52:20,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,2018-05-18 10:25:53,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,2017-08-01 18:38:42,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,2017-08-10 21:48:40,cool_stuff


In [5]:
transaction.shape

(112650, 3)

In [6]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 3 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   order_id                       112650 non-null  object
 1   order_purchase_timestamp       112650 non-null  object
 2   product_category_name_english  111023 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


### Drop null

In [7]:
transaction.isnull().sum()

order_id                            0
order_purchase_timestamp            0
product_category_name_english    1627
dtype: int64

In [8]:
transaction.dropna()

,order_id,order_purchase_timestamp,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,2017-09-13 08:59:02,cool_stuff
1,130898c0987d1801452a8ed92a670612,2017-06-28 11:52:20,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,2018-05-18 10:25:53,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,2017-08-01 18:38:42,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,2017-08-10 21:48:40,cool_stuff
...,...,...,...
112645,ffebd80e3291e811c308365936897efd,2018-07-13 20:04:05,garden_tools
112646,ffee31fb4b5e35c9123608015637c495,2018-08-18 10:00:59,furniture_decor
112647,fff7c4452f050315db1b3f24d9df5fcd,2017-06-01 16:53:03,watches_gifts
112648,fffa82886406ccf10c7b4e35c4ff2788,2017-12-18 16:33:07,sports_leisure


In [9]:
transaction['product_category_name_english'].nunique()

71

In [10]:
most_sold = transaction['product_category_name_english'].value_counts()
most_sold

bed_bath_table               11115
health_beauty                 9670
sports_leisure                8641
furniture_decor               8334
computers_accessories         7827
                             ...  
arts_and_craftmanship           24
cds_dvds_musicals               14
la_cuisine                      14
fashion_childrens_clothes        8
security_and_services            2
Name: product_category_name_english, Length: 71, dtype: int64

## Category Rules

In [11]:
transaction_catg = pd.get_dummies(data = transaction, columns = ['product_category_name_english'], prefix = 'catg').groupby([
                    'order_id','order_purchase_timestamp']).sum().reset_index()
transaction_catg

,order_id,order_purchase_timestamp,catg_agro_industry_and_commerce,catg_air_conditioning,catg_art,catg_arts_and_craftmanship,catg_audio,catg_auto,catg_baby,catg_bed_bath_table,...,catg_security_and_services,catg_signaling_and_security,catg_small_appliances,catg_small_appliances_home_oven_and_coffee,catg_sports_leisure,catg_stationery,catg_tablets_printing_image,catg_telephony,catg_toys,catg_watches_gifts
0,00010242fe8c5a6d1ba2dd792cb16214,2017-09-13 08:59:02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00018f77f2f0320c557190d7a144bdd3,2017-04-26 10:53:06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000229ec398224ef6ca0657da4fc703e,2018-01-14 14:33:31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00024acbcdf0a6daa1e931b038114c75,2018-08-08 10:00:35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,2017-02-04 13:57:51,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,2018-04-23 13:57:06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98662,fffcd46ef2263f404302a634eb57f7eb,2018-07-14 10:26:46,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98663,fffce4705a9662cd70adb13d4a31832d,2017-10-23 17:07:56,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
98664,fffe18544ffabc95dfada21779c9644f,2017-08-14 23:02:59,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
transaction_catg.columns = transaction_catg.columns.str.replace('catg_', '')
transaction_catg = transaction_catg.drop(columns = 'order_purchase_timestamp')
transaction_catg.head(5)

,order_id,agro_industry_and_commerce,air_conditioning,art,arts_and_craftmanship,audio,auto,baby,bed_bath_table,books_general_interest,...,security_and_services,signaling_and_security,small_appliances,small_appliances_home_oven_and_coffee,sports_leisure,stationery,tablets_printing_image,telephony,toys,watches_gifts
0,00010242fe8c5a6d1ba2dd792cb16214,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00018f77f2f0320c557190d7a144bdd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000229ec398224ef6ca0657da4fc703e,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00024acbcdf0a6daa1e931b038114c75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
transaction_catg = transaction_catg.groupby('order_id').sum().reset_index()
for i in transaction_catg.columns[1:]:
    transaction_catg[i] = transaction_catg[i].apply(lambda x : 1 if x > 0 else 0)
transaction_catg.head(5)

,order_id,agro_industry_and_commerce,air_conditioning,art,arts_and_craftmanship,audio,auto,baby,bed_bath_table,books_general_interest,...,security_and_services,signaling_and_security,small_appliances,small_appliances_home_oven_and_coffee,sports_leisure,stationery,tablets_printing_image,telephony,toys,watches_gifts
0,00010242fe8c5a6d1ba2dd792cb16214,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00018f77f2f0320c557190d7a144bdd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000229ec398224ef6ca0657da4fc703e,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,00024acbcdf0a6daa1e931b038114c75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
frequent_catgsets = fpgrowth(transaction_catg.iloc[:,1:], min_support = 0.00002, use_colnames = True)
frequent_catgsets['length'] = frequent_catgsets['itemsets'].apply(lambda x : len(x))
frequent_catgsets.sort_values('support', ascending = False)

C:\Users\quoct\AppData\Roaming\Python\Python38\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
10,0.095443,(bed_bath_table),1
7,0.089555,(health_beauty),1
11,0.078244,(sports_leisure),1
18,0.067794,(computers_accessories),1
2,0.065362,(furniture_decor),1
...,...,...,...
135,0.000020,"(stationery, computers_accessories)",2
139,0.000020,"(stationery, watches_gifts)",2
140,0.000020,"(furniture_decor, stationery)",2
141,0.000020,"(furniture_decor, industry_commerce_and_business)",2


In [40]:
catg_rules = association_rules(frequent_catgsets, metric = 'lift', min_threshold = 1)
catg_rules = catg_rules[catg_rules.conviction <= 1.1]
catg_rules.sort_values('confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(home_confort),(bed_bath_table),0.004024,0.095443,0.000436,0.108312,1.134835,0.000052,1.014432
2,(construction_tools_safety),(home_construction),0.001693,0.004966,0.000020,0.011976,2.411487,0.000012,1.007095
4,(market_place),(books_general_interest),0.002838,0.005189,0.000020,0.007143,1.376479,0.000006,1.001968
0,(bed_bath_table),(home_confort),0.095443,0.004024,0.000436,0.004566,1.134835,0.000052,1.000545
3,(home_construction),(construction_tools_safety),0.004966,0.001693,0.000020,0.004082,2.411487,0.000012,1.002399
5,(books_general_interest),(market_place),0.005189,0.002838,0.000020,0.003906,1.376479,0.000006,1.001073
7,(fashion_bags_accessories),(fashion_childrens_clothes),0.018892,0.000081,0.000020,0.001073,13.233101,0.000019,1.000993


In [41]:
catg_rules.to_csv(path + '\Web App\Data\catg_rules.csv', index = False)

<>:1: DeprecationWarning: invalid escape sequence \W
<>:1: DeprecationWarning: invalid escape sequence \W
C:\Users\quoct\AppData\Local\Temp\ipykernel_19988\473627688.py:1: DeprecationWarning: invalid escape sequence \W
  catg_rules.to_csv(path + '\Web App\Data\catg_rules.csv', index = False)


## Product Rules

In [87]:
order_data['product_id'].nunique()

32951

In [88]:
def get_prod_list(row):
    tempList = []
    for prod in row.unique() : 
        tempList.append(prod)
    return tempList
prod_list = order_data[['order_id', 'product_id']].groupby('order_id').agg(get_prod_list).reset_index().product_id
prod_list = list(prod_list)
prod_list[:5]

[['4244733e06e7ecb4970a6e2683c13e61'],
 ['e5f2d52b802189ee658865ca93d83a8f'],
 ['c777355d18b72b67abbeef9df44fd0fd'],
 ['7634da152a4610f1595efa32f14722fc'],
 ['ac6c3623068f30de03045865e4e10089']]

In [89]:
tEncoder = TransactionEncoder()
trans_array = tEncoder.fit(prod_list).transform(prod_list)
trans_prod = pd.DataFrame(trans_array, columns = tEncoder.columns_)
trans_prod.head(5)

,00066f42aeeb9f3007548bb9d3f33c38,00088930e925c41fd95ebfe695fd2655,0009406fd7479715e4bef61dd91f2462,000b8f95fcb9e0096488278317764d19,000d9be29b5207b54e86aa1b1ac54872,0011c512eb256aa0dbbb544d8dffcf6e,00126f27c813603687e6ce486d909d01,001795ec6f1b187d37335e1c4704762e,001b237c0e9bb435f2e54071129237e9,001b72dfd63e9833e8c02742adf472e3,...,ffef256879dbadcab7e77950f4f4a195,fff0a542c3c62682f23305214eaeaa24,fff1059cd247279f3726b7696c66e44e,fff28f91211774864a1000f918ed00cc,fff515ea94dbf35d54d256b3e39f0fea,fff6177642830a9a94a0f2cba5e476d1,fff81cc3158d2725c0655ab9ba0f712c,fff9553ac224cec9d15d49f5a263411f,fffdb2d0ec8d6a61f0a0a0db3f25b441,fffe9eeff12fcbd74a2f2b007dde0c58
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [90]:
frequent_prodsets = fpgrowth(trans_prod, min_support = 0.00003, use_colnames = True)
frequent_prodsets['length'] = frequent_prodsets['itemsets'].apply(lambda x : len(x))
frequent_prodsets

,support,itemsets,length
0,0.000091,(4244733e06e7ecb4970a6e2683c13e61),1
1,0.000030,(c777355d18b72b67abbeef9df44fd0fd),1
2,0.000122,(ac6c3623068f30de03045865e4e10089),1
3,0.000051,(ef92defde845ab8450f9d70c526ef70f),1
4,0.000030,(8d4f2bb7e93e6710a28f34fa83ee7d28),1
...,...,...,...
7985,0.000030,"(fc5dd987f12a7b823a76a44aa1ba88f6, 7eaf66aca1e...",2
7986,0.000030,"(100260bf9206d859dc8db128c8c5afc9, a2a91fe6d87...",2
7987,0.000061,"(5d790355cbeded0cd60e25cbc4c527a2, 5fc3e6a4b52...",2
7988,0.000030,"(e2cac69b319c0f8a21dbf04b925121bf, b9900407a55...",2


In [91]:
prod_rules = association_rules(frequent_prodsets, metric = 'lift', min_threshold = 1)
prod_rules = prod_rules[prod_rules.confidence >= 0.2]
prod_rules = prod_rules[prod_rules.conviction <= 1.4]
prod_rules.sort_values('confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(1427b126f61597524866770b05d4eed2),(5526b1ae9ab2688cf600783cece249df),0.000142,0.000223,0.000041,0.285714,1281.376623,0.000041,1.399688
101,(5b8a5a9417210b1b84b67b9a7aefb935),(e5ae72c62ebfa708624f5029d609b160),0.000213,0.000345,0.000061,0.285714,829.126050,0.000061,1.399518
62,(2d27434c710806b971a721da337a112a),(86c43103446290e7efacad3701cd654d),0.000111,0.000274,0.000030,0.272727,996.626263,0.000030,1.374624
124,(3cc4b421c1f28dd22e86fc4aaafb600e),(42a2bd596fda1baef5719cb74f73030c),0.000111,0.000213,0.000030,0.272727,1281.376623,0.000030,1.374707
57,(ee57070aa3b24a06fdd0e02efd2d757d),(0d85c435fd60b277ffb9e9b0f88f927a),0.000233,0.000639,0.000061,0.260870,408.554865,0.000061,1.352077
75,(f2ecbb02b6da187a9f14b8ac5781faec),(017692475c1c954ff597feda05131d73),0.000122,0.000243,0.000030,0.250000,1027.770833,0.000030,1.333009
84,(060f0122cdc8f25db4a0d7359fdae7b0),(57f2bc497c1a3ebe41ba7a06d78ed159),0.000162,0.000385,0.000041,0.250000,649.118421,0.000040,1.332820
108,(ad0a798e7941f3a5a2fb8139cb62ad78),(b0c89945c034268074f5f80b362bda34),0.000122,0.000162,0.000030,0.250000,1541.656250,0.000030,1.333117
117,(6d2fde7d12bb6ff367dbda120ba8828e),(fb29f48bfea41db52e349454f433340e),0.000122,0.000081,0.000030,0.250000,3083.312500,0.000030,1.333225
37,(060cb19345d90064d1015407193c233d),(98d61056e0568ba048e5d78038790e77),0.000253,0.000385,0.000061,0.240000,623.153684,0.000061,1.315283


In [92]:
prod_rules.to_csv(path + '\Web App\Data\prod_rules.csv', index = False)

<>:1: DeprecationWarning: invalid escape sequence \W
<>:1: DeprecationWarning: invalid escape sequence \W
C:\Users\quoct\AppData\Local\Temp\ipykernel_5608\3699731390.py:1: DeprecationWarning: invalid escape sequence \W
  prod_rules.to_csv(path + '\Web App\Data\prod_rules.csv', index = False)
